In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
from sklearn.preprocessing import MinMaxScaler
import datetime

In [2]:
combined_data = pd.read_csv('../data/intermediates/pre_acpca.csv')
combined_data.set_index('FIPS', inplace=True)
combined_data.head()

,Density per square mile of land area - Population,Rural-urban_Continuum Code_2013,Percent of adults with less than a high school diploma 2014-18,Percent of adults with a bachelor's degree or higher 2014-18,Unemployment_rate_2018,Med_HH_Income_Percent_of_State_Total_2018,Jan Temp AVG / F,Feb Temp AVG / F,Mar Temp AVG / F,Apr Temp AVG / F,...,SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!Elementary school (grades 1-8),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!High school (grades 9-12),SCHOOL ENROLLMENT!!Population 3 years and over enrolled in school!!College or graduate school,POP_ESTIMATE_2018,HospCt,NursingCt,Hispanic Population,Black Population,Deaths,Outbreak_date
FIPS,,,,,,,,,,,,,,,,,,,,,
1003,114.6,3.0,9.7,31.3,3.6,115.5,52.0,62.0,59.7,66.0,...,23254.0,11209.0,6503.0,218022,3,25,10131.0,19492.0,6,2020-04-20
1015,195.7,3.0,15.9,18.0,4.7,91.0,44.2,52.6,53.0,63.0,...,11224.0,4972.0,6890.0,114277,3,16,4469.0,24153.0,3,2020-04-20
1049,91.5,6.0,25.9,12.7,3.8,79.2,41.5,49.9,49.8,60.7,...,8313.0,4354.0,1654.0,71385,1,7,10595.0,1325.0,5,2020-05-19
1051,128.2,2.0,13.6,24.5,3.4,121.0,46.7,57.6,55.7,64.4,...,7403.0,5128.0,4022.0,81887,2,7,2467.0,17570.0,7,2020-05-04
1055,195.2,3.0,15.5,17.7,4.1,90.0,43.9,51.5,52.2,62.8,...,10215.0,4628.0,4019.0,102501,2,15,4116.0,16094.0,8,2020-04-04


In [3]:
# path = '/Users/beatrizmedeiros/Johns Hopkins/Fardin Ganjkhanloo - Number of trips/'

# for i in combined_data.index:
#     print(i)
#     outbreak_date = combined_data.loc[i, 'Outbreak_date']
#     trips = pd.read_csv(path + outbreak_date.strftime('%Y_%m_%d') + '.csv', index_col=0)
#     intra_mob = trips.loc[i, str(i)]
#     inter_mob = np.sum(trips.loc[i]) + np.sum(trips[str(i)])
#     print(intra_mob, inter_mob)
# #     print(trips.loc[i])
# #     print(trips[str(i)])
# #     print(trips.head())
#     break

In [4]:
path = '/Users/beatrizmedeiros/Johns Hopkins/Fardin Ganjkhanloo - Number of trips/'
sg_mobility = pd.DataFrame(index=combined_data.index, columns=['1 Month Prior Intra-Mobility', '1 Month Prior Inter-Mobility', '2wk Prior Intra-Mobility', '2wk Onset Intra-Mobility', '2wk Post Intra-Mobility', '2wk Prior Inter-Mobility', '2wk Onset Inter-Mobility', '2wk Post Inter-Mobility'])
# print(sg_mobility.head())
for i in combined_data.index: 
    print(i)
    inter_mob_per_day = np.zeros(48)
    intra_mob_per_day = np.zeros(48)
    row = combined_data.loc[i]
    outbreak_date = datetime.datetime.strptime(row['Outbreak_date'], '%Y-%m-%d')
#     print(outbreak_date)
#     outbreak_date = row.loc['Outbreak_date']
    # from 5 weeks pior to 2 weeks after 
    date1 = 34
    date2 = 14
    for j in range(-date1,date2): 
        mob_day = outbreak_date + datetime.timedelta(days=j) #.AddDays(j)
        mob_day = mob_day.date()
        try: 
            trips = pd.read_csv(path + mob_day.strftime('%Y_%m_%d') + '.csv', index_col=0)
        except: 
            print('missing mobility data for', mob_day.strftime('%Y_%m_%d'))
            if j > -date1: 
                inter_mob_per_day[j+date1] = inter_mob_per_day[j+date1-1]
                intra_mob_per_day[j+date1] = intra_mob_per_day[j+date1-1]
            else: 
                try: 
                    trips = pd.read_csv(path + outbreak_date.strftime('%Y_%m_%d') + '.csv', index_col=0)
                except: 
                    print('missing mobility data for outbreak date:', outbreak_date.strftime('%Y_%m_%d'))
                    continue
            continue
        intra_mob = trips.loc[i, str(i)]
        inter_mob = np.sum(trips.loc[i]) + np.sum(trips[str(i)]) - intra_mob # trips between this county and all OTHER counties 
        inter_mob_per_day[j+date1] = inter_mob
        intra_mob_per_day[j+date1] = intra_mob
    
    inter_prior_month_mob_arr = inter_mob_per_day[0:1+date1]
    inter_prior_mob_arr = np.array(inter_mob_per_day[date1:date1+14])
    inter_onset_mob_arr = np.array(inter_mob_per_day[date1-7:-7])
    inter_post_mob_arr = np.array(inter_mob_per_day[-14:])
    
    inter_prior_month_mob = np.mean(inter_prior_month_mob_arr)
    inter_prior_mob = np.mean(inter_prior_mob_arr)
    inter_onset_mob = np.mean(inter_onset_mob_arr)
    inter_post_mob = np.mean(inter_post_mob_arr)
    
    intra_prior_month_mob_arr = intra_mob_per_day[0:1+date1]
    intra_prior_mob_arr = np.array(intra_mob_per_day[date1:date1+14])
    intra_onset_mob_arr = np.array(intra_mob_per_day[date1-7:-7])
    intra_post_mob_arr = np.array(intra_mob_per_day[-14:])
    
    intra_prior_month_mob = np.mean(intra_prior_month_mob_arr)
    intra_prior_mob = np.mean(intra_prior_mob_arr)
    intra_onset_mob = np.mean(intra_onset_mob_arr)
    intra_post_mob = np.mean(intra_post_mob_arr)
    
    sg_mobility.loc[i,'1 Month Prior Inter-Mobility'] = inter_prior_month_mob
    sg_mobility.loc[i,'2wk Prior Inter-Mobility'] = inter_prior_mob
    sg_mobility.loc[i, '2wk Onset Inter-Mobility'] = inter_onset_mob
    sg_mobility.loc[i, '2wk Post Inter-Mobility'] = inter_post_mob
    
    sg_mobility.loc[i,'1 Month Prior Intra-Mobility'] = intra_prior_month_mob
    sg_mobility.loc[i,'2wk Prior Intra-Mobility'] = intra_prior_mob
    sg_mobility.loc[i, '2wk Onset Intra-Mobility'] = intra_onset_mob
    sg_mobility.loc[i, '2wk Post Intra-Mobility'] = intra_post_mob
    sg_mobility.to_csv('../data/processed/safegraph_mobility_avgs.csv')
#     if i < combined_data.index[5]:
#         print(mob_per_day)
#         print(prior_mob_arr, prior_mob)
#         print(onset_mob_arr, onset_mob)
#         print(pos_mob_arr, pos_mob)
sg_mobility.head()

1003
1015
1049
1051
1055
1069
1073
1081
1089
1095
1097
1101
1125
4001
4005
4013
4015
4017
4019
4021
4025
4027
5045
5069
5119
5143
6001
6013
6019
6025
6029
6037
missing mobility data for 2020_02_17
6041
6047
6053
6055
6059
6061
6065
missing mobility data for 2020_02_12
missing mobility data for 2020_02_13
missing mobility data for 2020_02_14
missing mobility data for 2020_02_15
missing mobility data for 2020_02_16
missing mobility data for 2020_02_17
6067
missing mobility data for 2020_02_14
missing mobility data for 2020_02_15
missing mobility data for 2020_02_16
missing mobility data for 2020_02_17
6071
6073
6075
6077
6081
6083
6085
missing mobility data for 2020_02_12
missing mobility data for 2020_02_13
missing mobility data for 2020_02_14
missing mobility data for 2020_02_15
missing mobility data for 2020_02_16
missing mobility data for 2020_02_17
6089
6095
6097
6099
6107
6111
6113
8001
8005
8013
8014
8031
8035
8041
missing mobility data for 2020_02_14
missing mobility data for 202

,1 Month Prior Intra-Mobility,1 Month Prior Inter-Mobility,2wk Prior Intra-Mobility,2wk Onset Intra-Mobility,2wk Post Intra-Mobility,2wk Prior Inter-Mobility,2wk Onset Inter-Mobility,2wk Post Inter-Mobility
FIPS,,,,,,,,
1003,33777.3,46481.7,35667.4,33594.9,35667.4,50168.3,44952.5,50168.3
1015,16525.4,24411.4,17253.9,16569.5,17253.9,26021.2,24522.1,26021.2
1049,9833.23,15213.2,10613,10474.5,10613,17658.8,17111.6,17658.8
1051,12356.5,21159,13449.1,13038.1,13449.1,25090.3,23720.9,25090.3
1055,18028.4,26428,15389.1,15663.1,15389.1,21208.9,21638.4,21208.9


In [5]:
sg_mobility.to_csv('../data/processed/safegraph_mobility_avgs.csv')